# Import

In [1]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml


# URL Rule

In [87]:
# comp_id = 380725    # 기업 고유 UID
p_num = 1             # 리뷰 페이지 (4 reviews / page)

### query로 CompID 가져오기

In [101]:
Keyword = '넷마블'

In [102]:
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(Keyword))
# print(search_req.text)

In [103]:
search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text

In [104]:
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

[<a href="/Comp/CompSummary/H79411">
                 넷마블
             </a>]

In [105]:
comp_id = re.findall('/([^"]*)"', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))[0].split('/')[2]
comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]

comp_id, comp_name

('H79411', '넷마블')

In [106]:
URL = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=0&isEmploy=false&jobCode='
print(URL)

https://www.catch.co.kr/api/v1.0/comp/reviewInfo/H79411/commentList?currentPage=1&pageSize=5000&isNew=false&employType=0&isEmploy=false&jobCode=


# Json to DataFrame 

In [107]:
# Json 형식의 웹 데이터 가져오기
raw_data = requests.get(URL).json()
# print(raw_data)

In [108]:
data = raw_data['reviewList']
data[0]

{'idx': 207128,
 'CompID': 'H79411',
 'CompName': '넷마블',
 'RegDate': '2023-06-08T10:46:41.900Z',
 'CI': 'H79411.jpg',
 'EmployText': '전직',
 'Gender2': 2,
 'RecomName': '추천',
 'EmployType': '계약직',
 'NewOld': '경력',
 'Answer': None,
 'Good': '눈치볼필요없다   복장자유롭고 자유로운분위기 \n 복리후생 굿   ',
 'Bad': '동종업계대비 낮은연봉 업무 스트레스 다소높음\n 고인물이 많음  ',
 'UsefulY': 0,
 'MyUsefulY': None,
 'MyOpinion': None,
 'CareerYear': 5,
 'CareerYearYN': 'Y',
 'Keyword1': None,
 'Keyword2': None,
 'Keyword3': None,
 'Keyword1YN': 'N',
 'Keyword2YN': 'N',
 'Keyword3YN': 'N',
 'JobName': '일반영업',
 'Area': '서울',
 'MyStarScore': 4}

In [109]:
df = pd.DataFrame(data)
df.head()

,idx,CompID,CompName,RegDate,CI,EmployText,Gender2,RecomName,EmployType,NewOld,...,CareerYearYN,Keyword1,Keyword2,Keyword3,Keyword1YN,Keyword2YN,Keyword3YN,JobName,Area,MyStarScore
0,207128,H79411,넷마블,2023-06-08T10:46:41.900Z,H79411.jpg,전직,2,추천,계약직,경력,...,Y,None,None,None,N,N,N,일반영업,서울,4.0
1,206247,H79411,넷마블,2023-06-04T05:35:31.877Z,H79411.jpg,현직,2,비추,정규직,신입,...,N,None,None,None,N,N,N,포장/가공,서울,4.0
2,201336,H79411,넷마블,2023-05-22T06:04:25.730Z,H79411.jpg,전직,2,추천,프리랜서,신입,...,Y,None,None,None,N,N,N,그래픽디자인/CG,서울,3.8
3,200208,H79411,넷마블,2023-05-20T08:03:40.137Z,H79411.jpg,현직,2,추천,정규직,신입,...,N,None,None,None,N,N,N,게임,서울,5.0
4,200105,H79411,넷마블,2023-05-20T04:04:21.673Z,H79411.jpg,전직,2,비추,계약직,경력,...,N,None,None,None,N,N,N,경리/회계/결산,서울,4.0


In [110]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'CI', 'EmployText', 'Gender2',
       'RecomName', 'EmployType', 'NewOld', 'Answer', 'Good', 'Bad', 'UsefulY',
       'MyUsefulY', 'MyOpinion', 'CareerYear', 'CareerYearYN', 'Keyword1',
       'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN',
       'JobName', 'Area', 'MyStarScore'],
      dtype='object')

In [111]:
df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 'CareerYearYN',
            'MyUsefulY', 'RecomName', 'MyOpinion', 'CareerYear', 'Keyword1', 
            'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN', 'Area'], 
            axis=1, inplace=True)

df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'Good', 'Bad',
       'JobName', 'MyStarScore'],
      dtype='object')

In [112]:
# 정규식으로 이스케이프 문자 제거        # Good, Bad 리뷰 항목에만 적용.
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,207128,H79411,넷마블,2023-06-08T10:46:41.900Z,전직,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4.0
1,206247,H79411,넷마블,2023-06-04T05:35:31.877Z,현직,다양한 경험을 쌓을 수 있음. 조직문화가 딱딱하지 않아 적응이 쉽다.,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4.0
2,201336,H79411,넷마블,2023-05-22T06:04:25.730Z,전직,회사 내에서 자유롭고 일하기도 편한 곳이다 상사와도 편안한 관계로 지낼 수 있고 눈...,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,3.8
3,200208,H79411,넷마블,2023-05-20T08:03:40.137Z,현직,눈치볼 필요없이 칼퇴근가능하다. 워라벨이 아주 좋다. 만족,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5.0
4,200105,H79411,넷마블,2023-05-20T04:04:21.673Z,전직,네임밸류와 자유로운 것이 장점 나름 인천에서 출근하기 편함,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4.0
5,197115,H79411,넷마블,2023-05-16T06:58:49.637Z,전직,대기업. 초봉 높은 편. 프로그래머는 연봉 잘 줌. 복지포인트 나옴. 여러 프로젝트...,"맨날 사람 자르고 동료들 팀 바꿔 대고, 10분 단위로 사람 감시함. 야근 엄청 많음",캐릭터/만화/애니,1.4
6,193234,H79411,넷마블,2023-04-08T04:11:35.247Z,현직,퍼블리셔다 보니 어느 게임 회사보다 다양한 프로젝트 수행 가능(사업부도 로테이션 종...,마케팅 부서는 업무 강도가 정말 정말 쎄다. 모든 산업군 다 합쳐도 넷마블 마케팅실...,마케팅/PR/분석,2.0
7,192458,H79411,넷마블,2023-03-13T08:26:19.983Z,현직,출퇴근 자유로움 연차사용 자유로움 건물이 깨끗해요(새건물),회사위치가 어느 역에서든 다 멀어요 걸어서 20분 고인물이 회사분위기 다 흐립니다,경리/회계/결산,1.0
8,192267,H79411,넷마블,2023-03-06T09:25:46.840Z,현직,자유로운 유연근무 개인주의 강해 회식이나 행사 거의 없음,"경영진이 성과나 회사의 방향성, 비전을 공유해주지 않음 성과급, 연봉 인상률 등의 ...",기획/전략/경영,3.4
9,185725,H79411,넷마블,2022-10-03T05:53:48.580Z,전직,눈치볼 필요가 없이 칼퇴가 가능하다. 워라벨이 가능하며 급여도 정기적으로 인상이 된...,가끔 야근이 필요하며 파벌이 있고 퇴근 후에도 게임 지식을 쌓아야 적응할 수 있다.,일반영업,2.4


In [113]:
# RegDate 컬럼의 데이터 - 문자열 9번째 까지 출력 (YYYYMMDD 형식의 날짜 표기)
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:8].strip())
df

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,207128,H79411,넷마블,20230608,전직,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4.0
1,206247,H79411,넷마블,20230604,현직,다양한 경험을 쌓을 수 있음. 조직문화가 딱딱하지 않아 적응이 쉽다.,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4.0
2,201336,H79411,넷마블,20230522,전직,회사 내에서 자유롭고 일하기도 편한 곳이다 상사와도 편안한 관계로 지낼 수 있고 눈...,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,3.8
3,200208,H79411,넷마블,20230520,현직,눈치볼 필요없이 칼퇴근가능하다. 워라벨이 아주 좋다. 만족,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5.0
4,200105,H79411,넷마블,20230520,전직,네임밸류와 자유로운 것이 장점 나름 인천에서 출근하기 편함,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4.0
5,197115,H79411,넷마블,20230516,전직,대기업. 초봉 높은 편. 프로그래머는 연봉 잘 줌. 복지포인트 나옴. 여러 프로젝트...,"맨날 사람 자르고 동료들 팀 바꿔 대고, 10분 단위로 사람 감시함. 야근 엄청 많음",캐릭터/만화/애니,1.4
6,193234,H79411,넷마블,20230408,현직,퍼블리셔다 보니 어느 게임 회사보다 다양한 프로젝트 수행 가능(사업부도 로테이션 종...,마케팅 부서는 업무 강도가 정말 정말 쎄다. 모든 산업군 다 합쳐도 넷마블 마케팅실...,마케팅/PR/분석,2.0
7,192458,H79411,넷마블,20230313,현직,출퇴근 자유로움 연차사용 자유로움 건물이 깨끗해요(새건물),회사위치가 어느 역에서든 다 멀어요 걸어서 20분 고인물이 회사분위기 다 흐립니다,경리/회계/결산,1.0
8,192267,H79411,넷마블,20230306,현직,자유로운 유연근무 개인주의 강해 회식이나 행사 거의 없음,"경영진이 성과나 회사의 방향성, 비전을 공유해주지 않음 성과급, 연봉 인상률 등의 ...",기획/전략/경영,3.4
9,185725,H79411,넷마블,20221003,전직,눈치볼 필요가 없이 칼퇴가 가능하다. 워라벨이 가능하며 급여도 정기적으로 인상이 된...,가끔 야근이 필요하며 파벌이 있고 퇴근 후에도 게임 지식을 쌓아야 적응할 수 있다.,일반영업,2.4


# For 문으로 만들어보기
    - 리뷰 데이터가 있는 페이지까지 반복

In [114]:
rv = []

for p_num in range(1, 52) :
    url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=4&isNew=false&employType=0&isEmploy=false&jobCode='
    rv.append(requests.get(url).json()['reviewList'])


data = list(itertools.chain.from_iterable(rv))
df = pd.DataFrame(data)


df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 
            'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area',
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
            axis=1, inplace=True)
df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())


df

,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,207128,H79411,넷마블,202306,전직,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4.0
1,206247,H79411,넷마블,202306,현직,다양한 경험을 쌓을 수 있음. 조직문화가 딱딱하지 않아 적응이 쉽다.,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4.0
2,201336,H79411,넷마블,202305,전직,회사 내에서 자유롭고 일하기도 편한 곳이다 상사와도 편안한 관계로 지낼 수 있고 눈...,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,3.8
3,200208,H79411,넷마블,202305,현직,눈치볼 필요없이 칼퇴근가능하다. 워라벨이 아주 좋다. 만족,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5.0
4,200105,H79411,넷마블,202305,전직,네임밸류와 자유로운 것이 장점 나름 인천에서 출근하기 편함,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4.0
5,197115,H79411,넷마블,202305,전직,대기업. 초봉 높은 편. 프로그래머는 연봉 잘 줌. 복지포인트 나옴. 여러 프로젝트...,"맨날 사람 자르고 동료들 팀 바꿔 대고, 10분 단위로 사람 감시함. 야근 엄청 많음",캐릭터/만화/애니,1.4
6,193234,H79411,넷마블,202304,현직,퍼블리셔다 보니 어느 게임 회사보다 다양한 프로젝트 수행 가능(사업부도 로테이션 종...,마케팅 부서는 업무 강도가 정말 정말 쎄다. 모든 산업군 다 합쳐도 넷마블 마케팅실...,마케팅/PR/분석,2.0
7,192458,H79411,넷마블,202303,현직,출퇴근 자유로움 연차사용 자유로움 건물이 깨끗해요(새건물),회사위치가 어느 역에서든 다 멀어요 걸어서 20분 고인물이 회사분위기 다 흐립니다,경리/회계/결산,1.0
8,192267,H79411,넷마블,202303,현직,자유로운 유연근무 개인주의 강해 회식이나 행사 거의 없음,"경영진이 성과나 회사의 방향성, 비전을 공유해주지 않음 성과급, 연봉 인상률 등의 ...",기획/전략/경영,3.4
9,185725,H79411,넷마블,202210,전직,눈치볼 필요가 없이 칼퇴가 가능하다. 워라벨이 가능하며 급여도 정기적으로 인상이 된...,가끔 야근이 필요하며 파벌이 있고 퇴근 후에도 게임 지식을 쌓아야 적응할 수 있다.,일반영업,2.4


# While 문으로 만들기 
    - 그 다음 페이지에 리뷰 데이터 개수가 0 이면 멈춤


In [115]:
# len(requests.get(url).json()['reviewList'])

In [118]:
# Catch 사이트에 검색을 하면, 최상위 검색결과의 comp_ui를 받아오기

comp = '넷마블'
search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
# print(search_req.text)

search_soup = BeautifulSoup(search_req.text, 'lxml')
# search_soup.text
search_soup.select('li:nth-child(1) > div.txt > p.name > a')

comp_id = re.findall('/([^"]*)"', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))[0].split('/')[2]
comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]

comp_id, comp_name

('H79411', '넷마블')

In [119]:
# 위에서 받아온 comp_id를 가지고 데이터 받아와서 데이터프레임으로 만들기

try : 
    # comp_id = 380725
    p_num = 1

    rv = []


    while True : 

        url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=2000&isNew=false&employType=0&isEmploy=false&jobCode='
        rv.append(requests.get(url).json()['reviewList'])

        p_num += 1

        if len(requests.get(url).json()['reviewList']) == 0 :
            break


    data = list(itertools.chain.from_iterable(rv))      # = 리스트 컴프리헨션
    df = pd.DataFrame(data)


    df.drop(['CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 'CareerYear', 
            'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area',
            'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
            axis=1, inplace=True)
            
    df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
    df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())
    


except :
    print("리뷰가 존재하지 않습니다.")


df


,idx,CompID,CompName,RegDate,EmployText,Good,Bad,JobName,MyStarScore
0,207128,H79411,넷마블,202306,전직,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4.0
1,206247,H79411,넷마블,202306,현직,다양한 경험을 쌓을 수 있음. 조직문화가 딱딱하지 않아 적응이 쉽다.,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4.0
2,201336,H79411,넷마블,202305,전직,회사 내에서 자유롭고 일하기도 편한 곳이다 상사와도 편안한 관계로 지낼 수 있고 눈...,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,3.8
3,200208,H79411,넷마블,202305,현직,눈치볼 필요없이 칼퇴근가능하다. 워라벨이 아주 좋다. 만족,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5.0
4,200105,H79411,넷마블,202305,전직,네임밸류와 자유로운 것이 장점 나름 인천에서 출근하기 편함,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4.0
5,197115,H79411,넷마블,202305,전직,대기업. 초봉 높은 편. 프로그래머는 연봉 잘 줌. 복지포인트 나옴. 여러 프로젝트...,"맨날 사람 자르고 동료들 팀 바꿔 대고, 10분 단위로 사람 감시함. 야근 엄청 많음",캐릭터/만화/애니,1.4
6,193234,H79411,넷마블,202304,현직,퍼블리셔다 보니 어느 게임 회사보다 다양한 프로젝트 수행 가능(사업부도 로테이션 종...,마케팅 부서는 업무 강도가 정말 정말 쎄다. 모든 산업군 다 합쳐도 넷마블 마케팅실...,마케팅/PR/분석,2.0
7,192458,H79411,넷마블,202303,현직,출퇴근 자유로움 연차사용 자유로움 건물이 깨끗해요(새건물),회사위치가 어느 역에서든 다 멀어요 걸어서 20분 고인물이 회사분위기 다 흐립니다,경리/회계/결산,1.0
8,192267,H79411,넷마블,202303,현직,자유로운 유연근무 개인주의 강해 회식이나 행사 거의 없음,"경영진이 성과나 회사의 방향성, 비전을 공유해주지 않음 성과급, 연봉 인상률 등의 ...",기획/전략/경영,3.4
9,185725,H79411,넷마블,202210,전직,눈치볼 필요가 없이 칼퇴가 가능하다. 워라벨이 가능하며 급여도 정기적으로 인상이 된...,가끔 야근이 필요하며 파벌이 있고 퇴근 후에도 게임 지식을 쌓아야 적응할 수 있다.,일반영업,2.4


# 컬럼명 및 데이터 변경

### 컬럼명 변경

In [120]:
df.columns

Index(['idx', 'CompID', 'CompName', 'RegDate', 'EmployText', 'Good', 'Bad',
       'JobName', 'MyStarScore'],
      dtype='object')

In [121]:
df.rename(columns=
            {'idx':'review_uid', 
            'CompID':'comp_uid',
            'CompName':'comp_name', 
            'RegDate':'review_date', 
            'EmployText':'is_office', 
            'Good':'review_pos', 
            'Bad':'review_neg', 
            'MyStarScore':'review_rate', 
            'JobName':'position'}, 
            inplace=True)

df.head(1)

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,207128,H79411,넷마블,202306,전직,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4.0


### 전/현직 여부를 Boolean 값으로 변경

In [122]:
df[['is_office']]

,is_office
0,전직
1,현직
2,전직
3,현직
4,전직
5,전직
6,현직
7,현직
8,현직
9,전직


In [123]:
df['is_office'] = df['is_office'].apply(lambda x: x.replace('현직', '1').strip() == '1')
df[['is_office']]

,is_office
0,False
1,True
2,False
3,True
4,False
5,False
6,True
7,True
8,True
9,False


### review_rate 의 값들을 전부 정수형으로 반올림

In [124]:
df['review_rate'].round(0).astype(int)

0     4
1     4
2     4
3     5
4     4
5     1
6     2
7     1
8     3
9     2
10    4
11    4
12    5
13    3
14    5
15    4
16    3
17    3
18    4
19    3
20    4
Name: review_rate, dtype: int32

In [125]:
df['review_rate'] = df['review_rate'].round(0).astype(int)
df.head()

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,207128,H79411,넷마블,202306,False,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4
1,206247,H79411,넷마블,202306,True,다양한 경험을 쌓을 수 있음. 조직문화가 딱딱하지 않아 적응이 쉽다.,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4
2,201336,H79411,넷마블,202305,False,회사 내에서 자유롭고 일하기도 편한 곳이다 상사와도 편안한 관계로 지낼 수 있고 눈...,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,4
3,200208,H79411,넷마블,202305,True,눈치볼 필요없이 칼퇴근가능하다. 워라벨이 아주 좋다. 만족,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5
4,200105,H79411,넷마블,202305,False,네임밸류와 자유로운 것이 장점 나름 인천에서 출근하기 편함,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4


### 긍/부정 리뷰들을 한 컬럼에 합치기

In [126]:
a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

new_df = pd.concat([a, b])

new_df

,review_uid,comp_uid,comp_name,review_date,is_office,review_cont,position,review_rate
0,207128,H79411,넷마블,202306,False,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4
1,206247,H79411,넷마블,202306,True,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4
2,201336,H79411,넷마블,202305,False,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,4
3,200208,H79411,넷마블,202305,True,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5
4,200105,H79411,넷마블,202305,False,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4
5,197115,H79411,넷마블,202305,False,"맨날 사람 자르고 동료들 팀 바꿔 대고, 10분 단위로 사람 감시함. 야근 엄청 많음",캐릭터/만화/애니,1
6,193234,H79411,넷마블,202304,True,마케팅 부서는 업무 강도가 정말 정말 쎄다. 모든 산업군 다 합쳐도 넷마블 마케팅실...,마케팅/PR/분석,2
7,192458,H79411,넷마블,202303,True,회사위치가 어느 역에서든 다 멀어요 걸어서 20분 고인물이 회사분위기 다 흐립니다,경리/회계/결산,1
8,192267,H79411,넷마블,202303,True,"경영진이 성과나 회사의 방향성, 비전을 공유해주지 않음 성과급, 연봉 인상률 등의 ...",기획/전략/경영,3
9,185725,H79411,넷마블,202210,False,가끔 야근이 필요하며 파벌이 있고 퇴근 후에도 게임 지식을 쌓아야 적응할 수 있다.,일반영업,2


# DataFrame to CSV file

In [127]:
SAVE_PATH = r'./data/'

In [128]:
df['comp_name'][0]

'넷마블'

In [129]:
comp = df['comp_name'][0]

In [131]:
# 디렉토리 생성 (이미 존재하면 생성하지 않음)

os.makedirs(SAVE_PATH, exist_ok=True)

# csv 파일로 저장.

file_name = f"{comp}_catch.csv"
save_file_path = os.path.join(SAVE_PATH, file_name)

df.to_csv(save_file_path, index=False, encoding = "utf-8")

In [132]:
pd.read_csv(save_file_path)

,review_uid,comp_uid,comp_name,review_date,is_office,review_pos,review_neg,position,review_rate
0,207128,H79411,넷마블,202306,False,눈치볼필요없다 복장자유롭고 자유로운분위기 복리후생 굿,동종업계대비 낮은연봉 업무 스트레스 다소높음 고인물이 많음,일반영업,4
1,206247,H79411,넷마블,202306,True,다양한 경험을 쌓을 수 있음. 조직문화가 딱딱하지 않아 적응이 쉽다.,근무강도가 힘든 편이다. 혼자 다양한 일을 하다보니 업무가 과중하다,포장/가공,4
2,201336,H79411,넷마블,202305,False,회사 내에서 자유롭고 일하기도 편한 곳이다 상사와도 편안한 관계로 지낼 수 있고 눈...,개인적인 능력이 딸리면 일하기가 힘들고 대부분이 경력직이며 신입들은 거의 없는 수준...,그래픽디자인/CG,4
3,200208,H79411,넷마블,202305,True,눈치볼 필요없이 칼퇴근가능하다. 워라벨이 아주 좋다. 만족,보수적이고 수직적인 분위기로 경직되고 수동적인 업무형태,게임,5
4,200105,H79411,넷마블,202305,False,네임밸류와 자유로운 것이 장점 나름 인천에서 출근하기 편함,상사가 능력이 모자르고 편애가 심함 팀장 자격이 있어서보다는 할 사람이 없어서,경리/회계/결산,4
5,197115,H79411,넷마블,202305,False,대기업. 초봉 높은 편. 프로그래머는 연봉 잘 줌. 복지포인트 나옴. 여러 프로젝트...,"맨날 사람 자르고 동료들 팀 바꿔 대고, 10분 단위로 사람 감시함. 야근 엄청 많음",캐릭터/만화/애니,1
6,193234,H79411,넷마블,202304,True,퍼블리셔다 보니 어느 게임 회사보다 다양한 프로젝트 수행 가능(사업부도 로테이션 종...,마케팅 부서는 업무 강도가 정말 정말 쎄다. 모든 산업군 다 합쳐도 넷마블 마케팅실...,마케팅/PR/분석,2
7,192458,H79411,넷마블,202303,True,출퇴근 자유로움 연차사용 자유로움 건물이 깨끗해요(새건물),회사위치가 어느 역에서든 다 멀어요 걸어서 20분 고인물이 회사분위기 다 흐립니다,경리/회계/결산,1
8,192267,H79411,넷마블,202303,True,자유로운 유연근무 개인주의 강해 회식이나 행사 거의 없음,"경영진이 성과나 회사의 방향성, 비전을 공유해주지 않음 성과급, 연봉 인상률 등의 ...",기획/전략/경영,3
9,185725,H79411,넷마블,202210,False,눈치볼 필요가 없이 칼퇴가 가능하다. 워라벨이 가능하며 급여도 정기적으로 인상이 된...,가끔 야근이 필요하며 파벌이 있고 퇴근 후에도 게임 지식을 쌓아야 적응할 수 있다.,일반영업,2


# 함수화

In [133]:
import os
import sys
from datetime import datetime
import time

import re

import tqdm
import itertools

import json
import urllib.request

import pandas as pd
import numpy as np

import requests
from bs4 import BeautifulSoup
import lxml

In [134]:
SAVE_PATH = r'./data/'
# SAVE_PATH = r'/app/data/reviews/'     <--- for Docker

In [135]:
def get_review(comp = str, save = True): 
    
    search_url = 'https://www.catch.co.kr/Search/SearchList?Keyword={}'
    search_req = requests.get(search_url.format(comp)) # Keyword = comp_name
    # print(search_req.text)

    search_soup = BeautifulSoup(search_req.text, 'lxml')
    # search_soup.text
    search_soup.select('li:nth-child(1) > div.txt > p.name > a')

    comp_id = re.findall('/([^"]*)"', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a')))[0].split('/')[2]
    comp_name = re.findall('>([^"]*)<', re.sub(r'\s', '', str(search_soup.select('li:nth-child(1) > div.txt > p.name > a'))))[0]




    p_num = 1

    rv = []



    try : 

        while True : 

            url = f'https://www.catch.co.kr/api/v1.0/comp/reviewInfo/{comp_id}/commentList?currentPage={p_num}&pageSize=5000&isNew=false&employType=0&isEmploy=false&jobCode='
            rv.append(requests.get(url).json()['reviewList'])

            p_num += 1

            if len(requests.get(url).json()['reviewList']) == 0 :
                break


        data = list(itertools.chain.from_iterable(rv))
        df = pd.DataFrame(data)


        df.drop(['idx', 'CompID', 'CI', 'Gender2', 'EmployType', 'NewOld', 'Answer', 'UsefulY', 
                'RecomName', 'CareerYearYN', 'MyUsefulY', 'MyOpinion', 'Area', 'CareerYear',
                'Keyword1', 'Keyword2', 'Keyword3', 'Keyword1YN', 'Keyword2YN', 'Keyword3YN'], 
                axis=1, inplace=True)
        

        df['Good'] = df['Good'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())
        df['Bad'] = df['Bad'].apply(lambda x: re.sub(r'\s+', ' ', re.sub(r'\n+', ' ', x)).strip())

        df['RegDate'] = df['RegDate'].apply(lambda x: re.sub(r'[^0-9]', '', x)[:6].strip())

        df['EmployText'] = df['EmployText'].apply(lambda x: x.replace('현직', '1').strip() == '1')



        df.rename(columns=
           {'RegDate':'review_date', 
           'EmployText':'is_office', 
           'Good':'review_pos', 
           'Bad':'review_neg', 
           'MyStarScore':'review_rate', 
           'JobName':'position'
           }, inplace=True)


        df['review_rate'] = df['review_rate'].round(0).astype(int)



        # 긍/부정 리뷰들만 찢어서 합치기

        a = df.drop(['review_pos'], axis=1).rename(columns = {'review_neg':'review_cont'})
        b = df.drop(['review_neg'], axis=1).rename(columns = {'review_pos':'review_cont'})

        new_df = pd.concat([a, b])

        # return new_df




        # csv 파일로 저장.

        os.makedirs(SAVE_PATH, exist_ok=True)

        file_name = f"{comp_name}_catch.csv"
        save_file_path = os.path.join(SAVE_PATH, file_name)


        if save is True :  
            new_df.to_csv(save_file_path, index=False, encoding = "utf-8")
            print('결과를 저장했습니다.')
            return pd.read_csv(save_file_path)
        
        else : 
            print('결과가 저장되지 않습니다.')
            return new_df

        
    except : 
        print("리뷰가 존재하지 않습니다.")




In [136]:
get_review('펄어비스', False)

결과가 저장되지 않습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,펄어비스,202305,False,업무 스타일은 호불호가 갈릴 수 있으며 야근 시 야근비 따로 없음,마케팅/PR/분석,3
1,펄어비스,202212,True,회사 전반적으로 비전에 대한 공유가 아쉽고 경력 개발에 대한 기회가 부족하다고 생각함,기획/전략/경영,2
2,펄어비스,202207,False,"최고경영자는 바뀐지 얼마 안돼 모르겠고, 실장급 그이상 리더십들 중 정상적으로 경력...",기획/전략/경영,2
3,펄어비스,202103,False,야근이 매우 많음. 굳이 필요없는 야근도 많은 듯한 느낌이 듬. 고용 불안정,QA/테스트/검증,4
0,펄어비스,202305,False,정기적으로 명절 떡값이 제공되며 연령대가 높지 않고 연차 자유 사용,마케팅/PR/분석,3
1,펄어비스,202212,True,신사옥으로 이전하면서 휴게시설과 복지시설들은 만족스러운 편임,기획/전략/경영,2
2,펄어비스,202207,False,복지 좋다. 비포괄임금이다. 밥이 맛있는편이다. 주거지원비때문에 출퇴근이 편하다,기획/전략/경영,2
3,펄어비스,202103,False,"교통 편리, 직원 복지 및 사내 시설이 좋음 카페나 휴식 공간 등 쾌적해요.",QA/테스트/검증,4


In [29]:
get_review('화인석재')

리뷰가 존재하지 않습니다.


In [31]:
get_review('삼성전자')

,CompName,review_date,is_office,review_cont,position,review_rate
0,삼성전자,202308,True,수직적 조직문화 선택적 복리후생 불투명한 소통 기타등등,생산관리/공정관리/품질관리,4
1,삼성전자,202308,False,높은 업무강도와 특정기간 늘어나는 업무량 휴식시간이 필요,생산/제조/설비/조립,3
2,삼성전자,202306,False,정치질을 못한다면 일이 쏟아지고 무능력하지만 정치질 잘하는 상사와 비례로 뽑힌 여사...,웹개발,5
3,삼성전자,202306,True,위치 빼고 없다 역시 갓기업이다 너무 마음에 듭니다 갓삼성,일반영업,3
4,삼성전자,202306,False,워라밸 없음. 전혀 없음. 개인 생활 없음. 경직된 문화,기획/전략/경영,2
...,...,...,...,...,...,...
797,삼성전자,202009,False,정말 좋은 회사다. 글로벌 회사이며 겉잡을 수 없는 기술력이 독보적,판매/캐셔/매장관리,4
798,삼성전자,202009,True,보너스 굿!! 연차 사용 자유롭고 출퇴근도 자율,기획/전략/경영,4
799,삼성전자,202009,False,좋습니다. 대기업인 만큼 여러 성장할 점이 많아요.,재무/세무/IR,4
800,삼성전자,202009,False,사내 프로세스 및 일처리가 깨끗하고 깔끔하다. 다양한 직무를 경험할 수 있다.,네트워크/서버/보안,3


In [137]:
get_review('초록뱀미디어')

결과를 저장했습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,초록뱀미디어,202104,True,일도 제대로 안하고 위에 사람들은 무얼 하는지 모르겠다. 인간성 제로,영상/사진/촬영,1
1,초록뱀미디어,202104,True,좋은점이라고는 없다. 그냥 다녀보면 안다.. 답답 그자체,영상/사진/촬영,1


In [140]:
get_review('카카오', False)

결과가 저장되지 않습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,카카오,202307,True,경영진의 문제가 심각 일단 사이즈는 큰데 앞으로 뭐먹고..?,웹기획/PM/웹마케팅,4
1,카카오,202306,True,이쪽 특이긴 하지만 연봉 인상률이 낮음 자유로운 분위기가 독이 되는게 본인이 자율적...,응용프로그램개발,4
2,카카오,202306,True,진입장벽이 있고 업계 전반적으로 성장가능성이 높음. 연봉인상률 양호함. 부서에 따라...,빅데이터/AI,5
3,카카오,202305,False,업무강도가 있는편이고 개인적인 업무 편향으로 먼가 혼자 일한다,마케팅/PR/분석,5
4,카카오,202305,True,업무에 체계가 없어서 지시도 명확하지 않고..내가 무슨일을 하고 있는거지 하는 확신...,인사/노무/교육,4
...,...,...,...,...,...,...
75,카카오,202011,True,"수평적이고 자기주도적인 업무 문화, 자유로운 분위기, 출퇴근 자유",사무/총무/법무,3
76,카카오,202011,True,대기업이라 만족하고 좋아요 안정적이네요ㅎㅎ,통신기술/네트워크구축,4
77,카카오,202011,True,복지시스템이 엄청 잘되있어서 자존감이 높아져요. 복장자유가 너무 좋아요,마케팅/PR/분석,5
78,카카오,202010,False,성장하는 회사라서 들어가면 배울 것도 많고 안전보장,마케팅/PR/분석,4


In [141]:
from new_catch import get_review

In [143]:
get_review('크래프톤', False)

결과가 저장되지 않습니다.


,CompName,review_date,is_office,review_cont,position,review_rate
0,크래프톤,202305,True,사내 분위기와 꼰대들이 많다 업계 전반적으로 성장률이 낮다,그래픽디자인/CG,5
1,크래프톤,202305,False,리더쉽이 아쉽다. 배틀그라운드 흥하고 그 돈으로 뽑은 신입급들이 기존 리더들보다 더...,게임,4
2,크래프톤,202305,False,경영진이 타운홀미팅이고 뭐고 많이 하긴 하는데 지금 기업의 현황에 대해 실무자들이 ...,게임,3
3,크래프톤,202206,True,주가가 끝없이 추락하는데 주식 방어를 못하는 경영진에 대한 실망 ㅠㅠ,웹개발,5
4,크래프톤,202112,False,시대적 마인드가 다소 떨어지는 곳이다. 개선점을 잘 받아들이지 않고 정해져 있는 구...,게임,2
5,크래프톤,202112,True,"높은 업무강도, 초역세권을 버리고 애매한 위치로 옮겨진 본사",사무/총무/법무,4
6,크래프톤,202110,True,이 기업은 나쁜점은 없다고 생각하는데 아쉬운 점은 많습니다. 예를 들어서 근무시간이...,생산관리/공정관리/품질관리,3
7,크래프톤,202104,True,"경영진의 우유부단한 의사 결정, 타 회사에 비해 낮은 복지",DBA/데이터베이스,3
8,크래프톤,202011,True,팀 해체가 너무 잦아서 고용불안이 심한 편이다.,게임,3
0,크래프톤,202305,True,여름 휴가가 따로 부여되고 휴가 맘대로 쓸수 있다 업무가 끝나면 칼퇴가 가능하다,그래픽디자인/CG,5
